# Resolver el problema realizando un embedding para el campo text y KNN (No se incluye en los dos modelos previos)

In [3]:
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

DATA_DIR = Path('../data/nlp-getting-started')
TRAIN_PATH = DATA_DIR / 'train.csv'
TEST_PATH = DATA_DIR / 'test.csv'
RANDOM_SEED = 27

train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

# 1. Separar features y target
X = train_df['text'].fillna('').copy()
y = train_df['target'].copy()

# 2. Split estratificado train/validation (80/20)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y
)

X_train.head()


2721    Obama declares disaster for typhoon-devastated...
2259    Businesses are deluged with invzices. Make you...
1815    Neil_Eastwood77: I AM A KNOBHEAD!! Bin Laden f...
682     Morgan Silver Dollar 1880 S Gem BU DMPL Cameo ...
7216    @danagould @WaynesterAtl I agree with backgrou...
Name: text, dtype: object

In [ ]:

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knn', KNeighborsClassifier(algorithm='brute'))
])

param_grid = {
    # TF-IDF
    'tfidf__max_features': [3000, 5000, 8000],
    'tfidf__ngram_range': [(1,1), (1,2)],
    'tfidf__min_df': [1, 2, 3],
    'tfidf__norm': ['l2', None],        # normalización
    'tfidf__sublinear_tf': [True, False],  # log(1 + tf)
    'tfidf__smooth_idf': [True, False], 

    # KNN
    'knn__n_neighbors': [3, 5, 7, 11],
    'knn__metric': ['cosine', 'euclidean'],
    'knn__weights': ['uniform', 'distance'],
}


grid_search_knn = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    scoring='f1',   
    cv=3,
    n_jobs=-1,
    verbose=2
)

grid_search_knn.fit(X_train, y_train)

best_knn = grid_search_knn.best_estimator_


Fitting 3 folds for each of 2304 candidates, totalling 6912 fits
[CV] END knn__metric=cosine, knn__n_neighbors=3, knn__weights=uniform, tfidf__max_features=3000, tfidf__min_df=1, tfidf__ngram_range=(1, 1), tfidf__norm=l2, tfidf__smooth_idf=True, tfidf__sublinear_tf=True; total time=   0.4s
[CV] END knn__metric=cosine, knn__n_neighbors=3, knn__weights=uniform, tfidf__max_features=3000, tfidf__min_df=1, tfidf__ngram_range=(1, 1), tfidf__norm=l2, tfidf__smooth_idf=True, tfidf__sublinear_tf=True; total time=   0.5s
[CV] END knn__metric=cosine, knn__n_neighbors=3, knn__weights=uniform, tfidf__max_features=3000, tfidf__min_df=1, tfidf__ngram_range=(1, 1), tfidf__norm=l2, tfidf__smooth_idf=True, tfidf__sublinear_tf=False; total time=   0.5s
[CV] END knn__metric=cosine, knn__n_neighbors=3, knn__weights=uniform, tfidf__max_features=3000, tfidf__min_df=1, tfidf__ngram_range=(1, 1), tfidf__norm=l2, tfidf__smooth_idf=False, tfidf__sublinear_tf=True; total time=   0.5s
[CV] END knn__metric=cosine, 

In [7]:
y_val_pred = best_knn.predict(X_val)
val_f1 = f1_score(y_val, y_val_pred)

print(f'Validation F1 score: {val_f1:.4f}')

Validation F1 score: 0.7381


Nada mal para algo tan simple como KNN sobre los embeddings TF-IDF. Casi que alcanza al XGBoost. Me hace sentir mal por mi XGBoost... Quizás estoy haciendo algo bastante mal con las features (es probable).

In [5]:
from pathlib import Path

submissions_dir = Path('../resultados')
submissions_dir.mkdir(parents=True, exist_ok=True)
baseline_submit_path = submissions_dir / 'knn.csv'

# Transformer para texto (TF-IDF)
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    strip_accents='unicode',
    lowercase=True,
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english'
)

y_test_pred = best_knn.predict(test_df['text'].fillna(''))

submission = pd.DataFrame({
    'id': test_df['id'],
    'target': y_test_pred
})

# Guardar CSV
submission.to_csv(baseline_submit_path, index=False)